In [46]:
import os

import pandas as pd
import matplotlib as plt

def get_data(folder):
    """
    Reads in turnstile data from a specified folder in Data
    
    Input: turnstile data file i.e. 2016-2017_turnstile_data
    Output: a DataFrame with all rows from all files in folder
    """
    
    col_names = ['C/A',
                 'UNIT',
                 'SCP',
                 'STATION',
                 'LINENAME',
                 'DIVISION',
                 'DATE',
                 'TIME',
                 'DESC',
                 'ENTRIES',
                 'EXITS                                                               ']

    ## absolute path to Data folder
    data_dir = os.getcwd()+"/Data/" 
    
    return_df = pd.DataFrame(columns=col_names)
    for file in os.listdir(data_dir+folder):
        if not file.startswith('.'):
            file_path = "Data/"+folder+'/'+file
            return_df = pd.concat([return_df, pd.read_csv(file_path)],axis=0)
            
    return_df.rename(columns={return_df.columns[10]:'EXITS'},inplace=True)
    
    return(return_df)

In [47]:
df = get_data("2016-2017_turnstile_data")

In [48]:
df.shape

(5085918, 11)

In [8]:
# No missing values
df.isnull().sum()

C/A                                                                     0
UNIT                                                                    0
SCP                                                                     0
STATION                                                                 0
LINENAME                                                                0
DIVISION                                                                0
DATE                                                                    0
TIME                                                                    0
DESC                                                                    0
ENTRIES                                                                 0
EXITS                                                                   0
dtype: int64